In [ ]:
#See README file to see where code is sourced from.

In [ ]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
#from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
#from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.applications.resnet50 import preprocess_input

import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Model

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import scipy
import os
import random
import gc
from keras.callbacks import ModelCheckpoint
from zipfile import ZipFile
import glob
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
#Defining testing and training filepaths gathering emotion labels from file paths.
training_path = "/com.docker.devenvironments.code/fer2013new/trainimgs"
testing_path = "/com.docker.devenvironments.code/fer2013new/testimgs"
class_names = os.listdir(training_path)
class_names_testing = os.listdir(testing_path)

#Testing each path making sure all 7 labels are present.
print(class_names)
print(class_names_testing)

In [ ]:
training = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
testing = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)

In [ ]:
train_gen = training.flow_from_directory("/com.docker.devenvironments.code/fer2013new/trainimgs", target_size=(48,48), batch_size = 32, shuffle=True, class_mode='categorical')
test_gen = testing.flow_from_directory("/com.docker.devenvironments.code/fer2013new/testimgs", target_size=(48,48), batch_size = 32, shuffle=False, class_mode='categorical')

In [ ]:
#Identity Block

def identity(X, f, filters, stage, block):
    conv_name_base='res' + str(stage) + block + '_branch'
    bn_name_base='bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(filters=F2, kernel_size=(1, 1), strides=(1, 1), padding='same', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
#Convolutional Block

def convolutional (X, f, filters, stage, block, s=2):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
#ResNet50 Block

def ResNet50(input_shape=(48,48,3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


In [ ]:
resnet = ResNet50(input_shape=(48,48,3))

In [ ]:
#Transfer Learning to bypass most training since will take days.

newModel = resnet.output
newModel = Flatten()(newModel)
newModel = Dense(256, activation='relu', name='fc1',kernel_initializer=glorot_uniform(seed=0))(newModel)
newModel = Dense(128, activation='relu', name='fc2',kernel_initializer=glorot_uniform(seed=0))(newModel)
newModel = Dense(7,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(newModel)


In [ ]:
#Compile Model

model = Model(inputs=resnet.input, outputs=newModel)


In [ ]:
#resnet.load_weights("/com.docker.devenvironments.code/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [ ]:
#for layer in resnet.layers:
    #layer.trainable = False

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

In [ ]:
#Early Stopping to prevent Overfitting the model.

es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max')

#H = model.fit_generator(train_gen,validation_data=test_gen,epochs=100,verbose=1,callbacks=[mc,es])